In [1]:
##################################################################
#  Цель - отсканировать существующие препринты на сайте Банка России 
#     и собрать из этих них информацию о ключевых словах
# 1) Нужно научиться сканировать сайты и открывать пдф-файлы на этом сайте
# 2) Нужно научиться извлекать текст из этих pdf-файлов по заданному правилу
# 3) Нужно извлеченный текст соотнести с названием конкретного исследования в виде таблицы
# 4) Нужно экспортировать полученные данные в эксель формат
#
# Сайт 1: https://www.cbr.ru/ec_research/ser/
# Сайт 2: https://www.cbr.ru/dkp/system_p/
#################################################################

In [1]:
# библиотеки
import requests
import lxml
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# целевые сайты
url1 = 'https://www.cbr.ru/ec_research/ser/'
url2 = 'https://www.cbr.ru/dkp/system_p/research_notes/#a_119347'

r1 = requests.get(url1)
r2 = requests.get(url2)

# чтобы начать парсить, нужно преобразовать информацию с сайта в объект "Soup"
soup1 = BeautifulSoup(r1.text, 'html')
soup2 = BeautifulSoup(r2.text, 'html')

In [3]:
# находим все ссылки на исследования, которые расположены на сайте номер 1
base1 = soup1.find_all('a', class_ = 'referenceable')


In [4]:
# находим все ссылки на исследования, которые расположены на сайте номер 2
base2 = soup2.find_all('a')

In [5]:
# создаем словарь
research_data = []

In [6]:
# извлекаем ссылки и соответствующие названия к статьям из сайта номер 1
for link in base1:
    # извлекаем название и ссылку
    title_span = link.find('span', class_='document-regular_name_visible')
    if title_span:
        title = title_span.get_text(strip=True)
        href = link.get('href')

        research_data.append({
            'title': title,
            'link': f"https://www.cbr.ru{href}"
        })

In [7]:
# извлекаем ссылки и соответствующие названия к статьям из сайта номер 2
for link in base2:
    # извлекаем значение атрибута href из ссылки
    href = link.get('href', '')
    if href and href.startswith('/StaticHtml') or href.startswith('/Content') :
        title = link.get_text(strip=True)
        href = link.get('href')

        research_data.append({
            'title': title,
            'link': f"https://www.cbr.ru{href}"
        })

In [8]:
# можно посмотреть на отобранные статьи
for i, research in enumerate(research_data, 1):
    print(f"{i}. {research['title']}")
    print(f"Ссылка: {research['link']}")
    print()

1. Оценка разрыва и потенциального выпуска с учетом финансовых условий по данным опросов Банка России на примере ЦФО
Ссылка: https://www.cbr.ru/ec_research/ser/wp_155/

2. Учебная модель малой открытой экономики для анализа денежно-кредитной политики (с примерами из практики Банка России)
Ссылка: https://www.cbr.ru/ec_research/ser/wp_154/

3. Жесткости отечественного рынка труда в период структурной трансформации экономики: неизвестные возможности известного источника данных
Ссылка: https://www.cbr.ru/ec_research/ser/wp_153/

4. Неоднородность инфляционных ожиданий разных типов экономических агентов: выводы для денежно-кредитной политики
Ссылка: https://www.cbr.ru/ec_research/ser/wp_152/

5. О равновесиях в модели рынков депозитов с экзогенными издержками перехода вкладчиков из банка в банк
Ссылка: https://www.cbr.ru/ec_research/ser/wp_151/

6. Синхронизация деловых циклов России и Китая
Ссылка: https://www.cbr.ru/ec_research/ser/wp_150/

7. Декомпозиция индекса потребительских цен на 

### Теперь есть список статей и ссылки на них. Требуется из всех ссылок извлечь те, которые ведут на сайт и заменить их на ссылки на pdf файлы

In [ ]:
def replace_with_pdf(articles_list):
    # данная функция заменяет ссылки на статьи на прямые ссылки на pdf-файлы
    results = articles_list.copy() # финальный словарь, в который будут сохраняться финальные ссылки на pdf

    for i, article in enumerate(results):
        current_link = article['link'] # получаем текущую ссылку на статью
        print(f"Обрабатываем: {current_link}")

        if current_link.lower().endswith('.pdf'):
            print("Уже PDF, пропускаем")
            continue
        
        else:
            print("Уже исправляю ссылку на PDF")
            urltest = current_link
            rtest = requests.get(urltest)
            souptest = BeautifulSoup(rtest.text, 'html.parser')
            basetest = souptest.find_all('a')

            for i, link in enumerate(basetest, 1):
                href = link.get('href')
                if href and href.endswith('.pdf'):
                    if href.startswith('/'):
                        hrefpdf = f"https://www.cbr.ru{href}"
                        article['link'] = hrefpdf
                    elif href.startswith('http'):
                        hrefpdf = href
                        article['link'] = hrefpdf
                    else:
                        hrefpdf = f"https://www.cbr.ru/{href}"
                        article['link'] = hrefpdf

    seen_links = set()
    unique_research = []
    
    for research in results:
        link = research.get('link', '').strip()
        if link.endswith('.pdf'): # есть ссылки, которые ведут не на pdf, а на .xsls
            if link and link not in seen_links:
                seen_links.add(link)
                unique_research.append(research)
            elif not link:
                # Исследования без ссылки тоже добавляем
                unique_research.append(research)
        
    # Возвращаем список уникальных исследований
    return unique_research

research_data_pdf = replace_with_pdf(research_data)

Обрабатываем: https://www.cbr.ru/StaticHtml/File/180764/wp_155.pdf
Уже PDF, пропускаем
Обрабатываем: https://www.cbr.ru/StaticHtml/File/180505/wp_154.pdf
Уже PDF, пропускаем
Обрабатываем: https://www.cbr.ru/StaticHtml/File/178855/wp_153.pdf
Уже PDF, пропускаем
Обрабатываем: https://www.cbr.ru/StaticHtml/File/178678/wp_152.pdf
Уже PDF, пропускаем
Обрабатываем: https://www.cbr.ru/StaticHtml/File/177177/wp_151.pdf
Уже PDF, пропускаем
Обрабатываем: https://www.cbr.ru/StaticHtml/File/176714/wp_150.pdf
Уже PDF, пропускаем
Обрабатываем: https://www.cbr.ru/StaticHtml/File/175535/wp_149.pdf
Уже PDF, пропускаем
Обрабатываем: https://www.cbr.ru/StaticHtml/File/175520/wp_148.pdf
Уже PDF, пропускаем
Обрабатываем: https://www.cbr.ru/StaticHtml/File/175512/wp_147.pdf
Уже PDF, пропускаем
Обрабатываем: https://www.cbr.ru/StaticHtml/File/174669/wp_146.pdf
Уже PDF, пропускаем
Обрабатываем: https://www.cbr.ru/StaticHtml/File/172426/wp_145.pdf
Уже PDF, пропускаем
Обрабатываем: https://www.cbr.ru/StaticHtml

In [32]:
# можно посмотреть на отобранные статьи
for i, research in enumerate(research_data_pdf, 1):
    print(f"{i}. {research['title']}")
    print(f"Ссылка: {research['link']}")
    print()

1. Оценка разрыва и потенциального выпуска с учетом финансовых условий по данным опросов Банка России на примере ЦФО
Ссылка: https://www.cbr.ru/StaticHtml/File/180764/wp_155.pdf

2. Учебная модель малой открытой экономики для анализа денежно-кредитной политики (с примерами из практики Банка России)
Ссылка: https://www.cbr.ru/StaticHtml/File/180505/wp_154.pdf

3. Жесткости отечественного рынка труда в период структурной трансформации экономики: неизвестные возможности известного источника данных
Ссылка: https://www.cbr.ru/StaticHtml/File/178855/wp_153.pdf

4. Неоднородность инфляционных ожиданий разных типов экономических агентов: выводы для денежно-кредитной политики
Ссылка: https://www.cbr.ru/StaticHtml/File/178678/wp_152.pdf

5. О равновесиях в модели рынков депозитов с экзогенными издержками перехода вкладчиков из банка в банк
Ссылка: https://www.cbr.ru/StaticHtml/File/177177/wp_151.pdf

6. Синхронизация деловых циклов России и Китая
Ссылка: https://www.cbr.ru/StaticHtml/File/176714

## Извлекаем ключевые слова из PDF

In [14]:
import PyPDF2
from io import BytesIO
import re
import pdfplumber

In [16]:
response = requests.get('https://www.cbr.ru/Content/Document/File/115672/wp-62_e.pdf')
response.raise_for_status()
        
with pdfplumber.open(BytesIO(response.content)) as pdf:
    text = []
    for page in pdf.pages:
         text.append(page.extract_text())

print(text)

Cannot set gray non-stroke color because /'P17' is an invalid float value


['ЯНВАРЬ 2019\nMacroprudential Policy Efficiency:\nAssessment for the Uncollateralized\nConsumer Loans in Russia\nBank of Russia Working Paper Series No. 62\nNovember 2020\nKozlovtсeva I., Penikas H., Petreneva E., Ushakova Y.', 'Bank of Russia, Research and forecasting department\nIrina Kozlovtceva\nEmail: kozlovtsevaid@cbr.ru\nHenry Penikas\nEmail: penikasgi@mail.cbr.ru\nEkaterina Petreneva\nEmail: petrenevaea@cbr.ru\nYulia Ushakova\nEmail: ushakovayuv@cbr.ru\nWe acknowledge the feedback from the internal Bank of Russia research workshop participants,\nand specifically that of Konstantin Styrin. We wish to separately thank Andrey Sinyakov for bringing\nour attention to the issue that positive credit growth after restricting it for the developing economies\nimplies that it could have been much larger otherwise. We thank our colleagues from the Financial\nStability Department of the Bank of Russia who made valuable recommendations on the set of de-\npendent variables to investigate as 

### *Напишем функцию, которая извлекает текст из pdf-файла*

In [18]:
def extract_text_from_pdf(pdf_link: str) -> str:
    try:
        response = requests.get(pdf_link)
        response.raise_for_status()

        pdf_data = BytesIO(response.content)

    
        reader = PyPDF2.PdfReader(pdf_data, strict=False)
        pdf_text = []

        total_pages = len(reader.pages)
        
        # Извлекаем первые 5 страниц
        for page_num in range(min(5, total_pages)):
            content = reader.pages[page_num].extract_text()
            pdf_text.append(content)
        
        # Извлекаем последние 4 страницы (если есть)
        if total_pages > 5:  # Чтобы не дублировать страницы
            start_last_pages = max(5, total_pages - 4)  # Начинаем с max(5, total_pages-4)
            for page_num in range(start_last_pages, total_pages):
                content = reader.pages[page_num].extract_text()
                pdf_text.append(content)
            
        return pdf_text
    
    except:
        response = requests.get(pdf_link)
        response.raise_for_status()
                
        with pdfplumber.open(BytesIO(response.content)) as pdf:
            text = []
            for page in pdf.pages:
                text.append(page.extract_text())

In [ ]:
texttest = extract_text_from_pdf('https://www.cbr.ru/Content/Document/File/115672/wp-62_e.pdf')
print(texttest)

['ЯНВАРЬ 2019  \n  \n \n \n \n    \n    \nMacroprudential Policy Efficiency:  \nAssessment for the Uncollateralized \nConsumer Loans in Russia  \nBank of Russia Working Paper Series No. 62  \nNovember  2020 \n \nKozlovt сeva I., Penikas  H., Petreneva E., Ushakova Y. \n \n', 'Bank of Russia, Research and forecasting department\nIrina Kozlovtceva\nEmail: kozlovtsevaid@cbr.ru\nHenry Penikas\nEmail: penikasgi@mail.cbr.ru\nEkaterina Petreneva\nEmail: petrenevaea@cbr.ru\nYulia Ushakova\nEmail: ushakovayuv@cbr.ru\nWe acknowledge the feedback from the internal Bank of Russia research workshop participants,\nand speci\x0ccally that of Konstantin Styrin. We wish to separately thank Andrey Sinyakov for bringing\nour attention to the issue that positive credit growth after restricting it for the developing economies\nimplies that it could have been much larger otherwise. We thank our colleagues from the Financial\nStability Department of the Bank of Russia who made valuable recommendations on the

### *Напишем функцию, которая очищает текст*

In [23]:
def clean_keywords_text(text: str) -> str: # данная функция очищает и восстанавливает текст
    combined_text = "".join(text) # объединяем все страницы вместе
    lines = combined_text.split('\n') # разделяем текст на строки

    cleaned_lines = []
    for line in lines:
            # Убираем пробелы в начале и конце строки
            line = line.strip()
            cleaned_lines.append(line)

    full_text = ' '.join(cleaned_lines)

    full_text = re.sub(r'(\w+)-\s+', r'\1', full_text)
    full_text = re.sub(r'\s+', ' ', full_text)
    full_text = re.sub(r'\s+:', ':', full_text) # убираем пробел перед двоеточием
    full_text = re.sub(r'\s+ь', 'ь', full_text) # убираем пробел мягким знаком (в некоторых файлах при извлечении текста есть данная проблема)
    full_text = re.sub(r'\s+ы', 'ы', full_text)
    full_text = re.sub(r'\s+ъ', 'ъ', full_text)
    full_text = re.sub(r'\s+-', '-', full_text)
    full_text = re.sub(r'\s+»', '»', full_text)
    full_text = re.sub(r'\s+;', ';', full_text)
    full_text = re.sub(r':(\w)', r': \1', full_text) # добавляем пробел после двоеточия, если его нет

    return full_text.strip()

### *Напишем функцию, которая ищет в тексте ключевые слова*

In [24]:
def extract_keywords_intelligent(text: str) -> str: # извлечение ключевых слов с учетом различных форматов:
    if text == 'Битый текст':
        return 'Битый текст'
    else:
        # варианты паттернов для поиска начала ключевых слов
        patterns = [
            r'Ключевые слова:*',
            r'Keywords:*',
            r'Key words:*',
            r'KEYWORDS:*',
            r'KEY WORDS:*',
            r'Ключевые слова\s*',  # Без двоеточия
            r'Keywords\s*',        # Без двоеточия
            r'Key words\s*',       # Без двоеточия
        ]
        
        for pattern in patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                start_pos = match.end()
                # Находим позицию начала ключевых слов
                keywords_text = text[start_pos:]
                
                # Разбиваем текст на токены для анализа
                tokens = []
                current_token = ""
                
                # Проходим по каждому символу для анализа структуры
                for char in keywords_text:
                    if char in [':', '.']:  # Сначала проверяем точку и двоеточие (если они есть, то значит ключевые слова 100% уже перечислены)
                        if current_token:
                            tokens.append(current_token)  # Добавляем последнее слово
                        break  # Выходим из цикла

                    elif char in ['ь', 'ы', 'ъ']:
                        if current_token:
                            tokens.append(current_token)
                            current_token = ""
                        tokens.append(char)

                    elif char in [',', ';'] or char.isspace():
                        if current_token:
                            tokens.append(current_token)
                            current_token = ""
                        tokens.append(char)
                    else:
                        current_token += char


                # Собираем ключевые слова до условия остановки
                result_tokens = []
                stop = False
                prev_comma = False  # Флаг, что предыдущий токен был запятой
                
                for i, token in enumerate(tokens):
                    if stop:
                        break
                    
                    if (token[0].isupper() if token else False) and prev_comma:
                        # Слово с заглавной буквы после запятой - вероятно начало нового раздела
                        # Проверяем, не является ли это частью ключевого слова (например, "ВВП")
                        if len(token) > 1 and not any(c.islower() for c in token[1:]):
                            # Если все остальные буквы заглавные (акроним), то продолжаем
                            result_tokens.append(token)
                        else:
                            # Останавливаемся на слове с заглавной буквы после запятой
                            break
                            
                    elif token.lower().startswith('jel') or token.lower().startswith('j.e.l'):
                        # начало JEL классификации
                        break
                        
                    else:
                        result_tokens.append(token)
                    
                    # Обновляем флаг предыдущей запятой
                    prev_comma = (token == ',')
                
                # Собираем результат
                keywords_result = "".join(result_tokens).strip()
                
                # Очищаем результат от лишних символов в конце
                keywords_result = re.sub(r'[,\s]+$', '', keywords_result)  # Убираем запятые и пробелы в конце
                keywords_result = re.sub(r'\s+,\s*', ', ', keywords_result)  # Нормализуем запятые
                
                return keywords_result
            
        return 'Нет ключевых слов'

## Применим функции к полученным ранее ссылкам

In [25]:
extracted_text = extract_text_from_pdf('https://www.cbr.ru/Content/Document/File/115672/wp-62_e.pdf')

In [28]:
clean_text = clean_keywords_text(extracted_text)
keywords = extract_keywords_intelligent(clean_text) 
print(clean_text)

ЯНВАРЬ 2019 Macroprudential Policy Efficiency: Assessment for the Uncollateralized Consumer Loans in Russia Bank of Russia Working Paper Series No. 62 November 2020 Kozlovt сeva I., Penikas H., Petreneva E., Ushakova Y. Bank of Russia, Research and forecasting department Irina Kozlovtceva Email: kozlovtsevaid@cbr.ru Henry Penikas Email: penikasgi@mail.cbr.ru Ekaterina Petreneva Email: petrenevaea@cbr.ru Yulia Ushakova Email: ushakovayuv@cbr.ru We acknowledge the feedback from the internal Bank of Russia research workshop participants, and speci cally that of Konstantin Styrin. We wish to separately thank Andrey Sinyakov for bringing our attention to the issue that positive credit growth after restricting it for the developing economies implies that it could have been much larger otherwise. We thank our colleagues from the Financial Stability Department of the Bank of Russia who made valuable recommendations on the set of dependent variables to investigate as a target for the macroprude

In [29]:
def final_keywords_formation(data):
    for i, research in enumerate(data, 1):
        extracted_text = extract_text_from_pdf(research['link'])
        clean_text = clean_keywords_text(extracted_text)
        keywords = extract_keywords_intelligent(clean_text) 
        research['keywords_total'] = keywords #список всех ключевых слов

        keyword_list = [kw.strip() for kw in re.split(r'[,;]', keywords) if kw.strip()] # разделяю список на отдельные ключевые слова
        # добавляю каждое ключевое слово в отдельный столбец
        for j, keyword in enumerate(keyword_list, 1):
            research[f'keyword_{j}'] = keyword
        

        print(f"Препринт {i}: {keywords}")

In [33]:
finally_keywords = final_keywords_formation(research_data_pdf)

Препринт 1: разрыв выпуска, потенциальный выпуск, валовой региональный продукт, HP-фильтр, фильтр Калмана, факторная декомпозиция, региональный анализ
Препринт 2: денежно-кредитная политика, малая открытая экономика, валютный рынок, таргетирование инфляции, дилемма денежно-кредитной политики, графическая модель общего равновесия, квартальная прогнозная модель, Банк России
Препринт 3: постоянная эластичность замещения, CES, денежно-кредитная политика, ДКП, рынок труда
Препринт 4: инфляционные ожидания, неоднородные агенты, шоки ожиданий, денежно-кредитная политика, финансовые посредники, новая кейнсианская модель, общее равновесие, неоднородность инфляционных ожиданий
Препринт 5: банки, депозиты, издержки перехода, равновесие Нэша, равновесие в безопасных стратегиях, уровень благосостояния
Препринт 6: Россия, Китай, международные деловые циклы, торговые связи, OECD TiVA, Global VAR
Препринт 7: инфляция, деловой цикл, циклическая инфляция, ациклическая инфляция, кривая Филлипса
Препринт 

# Импортируем результаты в CSV

In [ ]:
df = pd.DataFrame(finally_keywords)
df.to_csv('Preprints_Links_keywords', index=False, encoding='utf-8-sig')
print("Данные экспортированы в Preprints_Links_keywords")